In [ ]:
1. Get all reviews
2. df1 - reviews > 5 votes
3. df2 - reviews > 10 votes
4. df3 - reviews (> 5 votes ) & (helpfulness < 0.5) + (> 10 votes ) & (helpfulness > 0.9)

In [45]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import gzip


from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import swifter
import textstat
from textstat import flesch_reading_ease

## Get All Reviews

In [3]:


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')



In [4]:
df = getDF('reviews_Electronics_5.json.gz')

KeyboardInterrupt: 

In [ ]:
def num_votes(arr) :
    return arr[1]

In [ ]:
MIN_VOTES = 5

In [ ]:
df1 = df[df['helpful'].apply(num_votes) >= MIN_VOTES]

In [ ]:
1. Get reviews of  helpfulness less than 0.5  -> A
2. Get reviews of  no of votes greater or equal to 10 -> B
3. Get review of helpfulness greater than or equal to 0.9 -> C
4. Tag A -> 0 
5. Tag C -> 1
6. Merge A + C
7. Bulid Model
8. Predict
9. Assign Values to train and test data : X
    
Until Step 6 Done in:  User_Features_All_Reviews

In [2]:
df_Cl = pd.read_csv('Classification_Data.csv')

In [3]:
df_Cl.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,helpfulness_score,is_helpful
0,A38FGQVJM18OWV,0972683275,"George S. Mitchell ""gsmitchell""","[8, 18]",I checked around Amazon as well as some other ...,1.0,What a piece of junk!,1291161600,"12 1, 2010",0.444444,0
1,A11ZP93TJED4T6,0972683275,Ryan,"[1, 8]",The fact that the pivot point on this TV mount...,2.0,poor manufacture quality..,1285891200,"10 1, 2010",0.125000,0
2,A1A5KUIIIHFF4U,1400501466,"Anonymous ""Anonymous""","[4, 9]",Have no idea why people love this thing so muc...,1.0,This thing is a junk kids toy.,1332547200,"03 24, 2012",0.444444,0
3,ASHQ1JCA241D6,1400501520,"Loraine Arnold ""Loraine""","[2, 5]",I bought the Nook primarily for reading but wa...,5.0,Great tablet and ereader,1361145600,"02 18, 2013",0.400000,0
4,A1889S56VFLVMZ,1400501520,Price John Monroe,"[2, 9]","Why pay to be restricted, I simply didn't see ...",1.0,"Over-priced, lacking in utility.",1369094400,"05 21, 2013",0.222222,0


In [4]:
len(df_Cl)

60804

In [26]:
def review_length(s) :
    return len(s.split())

In [7]:
df2 = df_Cl.dropna(axis=0)

In [8]:
len(df2)

59989

In [9]:
df_Cl['reviewText'].head()

0    I checked around Amazon as well as some other ...
1    The fact that the pivot point on this TV mount...
2    Have no idea why people love this thing so muc...
3    I bought the Nook primarily for reading but wa...
4    Why pay to be restricted, I simply didn't see ...
Name: reviewText, dtype: object

In [10]:
df2['review_length'] = df2['reviewText'].apply(review_length)

C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
X = df2[['overall','review_length']]

In [12]:
X.head()

,overall,review_length
0,1.0,138
1,2.0,77
2,1.0,112
3,5.0,101
4,1.0,54


In [13]:
y = df2['is_helpful']
	
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

In [14]:
X_train[:10]

,overall,review_length
9506,5.0,303
36779,5.0,165
8097,4.0,52
46458,5.0,480
22579,4.0,56
221,1.0,93
15968,1.0,117
9660,2.0,113
7081,1.0,167
12000,1.0,22


In [15]:
xgb_cl = xgb.XGBClassifier()

In [16]:
xgb_cl.fit(X_train,y_train, verbose=True)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [17]:
predictions=xgb_cl.predict(X_test)

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
print(confusion_matrix(y_test,predictions))

[[ 7513  2322]
 [ 1956 12205]]


In [24]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.79      0.76      0.78      9835
          1       0.84      0.86      0.85     14161

avg / total       0.82      0.82      0.82     23996



### Analysis on Test Data

In [34]:
test_data = pd.read_csv('Test_Data.csv')

In [36]:
len(test_data)

32510

In [37]:
test_data.head()

,reviewerID,asin,reviewerName,reviewText,overall,summary,unixReviewTime,reviewTime,helpfulness_score,user_deviation,review_delay,stem_words,lem_words,stem_sim_words,lem_sim_words,helpful,real_index
0,A20DDH4NT6Q1E8,B0036AWR8E,Koala1356,If you're looking at the Lowepro SlingShot cam...,5.0,"Versatile, Quality Backpack",1271116800,"04 13, 2010",0.931034,0.551259,950400,"['your', 'look', 'lowepro', 'slingshot', 'came...","['youre', 'looking', 'lowepro', 'slingshot', '...",['x'],"['camera', 'front', 'compartment', 'cloth', 'c...","[54, 58]",715950
1,A3V2P1ZV2LETF3,B000TXEE14,Artemaria,I bought this drive because it was on sale fro...,4.0,Good Budget Drive,1196208000,"11 28, 2007",0.975610,0.681306,0,"['bought', 'drive', 'sale', 'circuit', 'citi',...","['bought', 'drive', 'sale', 'circuit', 'city',...",[],"['mind', 'drive', 'storage', 'dont', 'cap', 'w...","[80, 82]",313454
2,AYU6VDCDO0542,B0001DA0CO,William N. Ostrove,I have recently become interested in digital p...,5.0,Great Camera for Beginners,1101600000,"11 28, 2004",0.909091,0.373805,13305600,"['recent', 'becom', 'interest', 'digit', 'phot...","['recently', 'become', 'interested', 'digital'...",[],"['camera', 'easyshare', 'software', 'auto', 'p...","[20, 22]",88450
3,A23O3QXCYT55JJ,B00BU7TV6G,i7Rage,SUMMARY: works beautifully to transfer VHS tap...,4.0,Not for editing,1291161600,"12 1, 2010",1.000000,0.954457,4147200,"['summari', 'work', 'beauti', 'transfer', 'vh'...","['summary', 'work', 'beautifully', 'transfer',...",[],"['recorder', 'home']","[61, 61]",1574701
4,A1WMHFPD9YLVFV,B0000B35CK,Abester,This card works perfectly in my 8 megapixel Ol...,5.0,Works great in my Olympus C-8080 8 mega pixel ...,1089763200,"07 14, 2004",1.000000,0.973060,10281600,"['card', 'work', 'perfectli', '8', 'megapixel'...","['card', 'work', 'perfectly', '8', 'megapixel'...",[],[],"[14, 14]",76480


In [29]:
def is_helpful(score) :
    if score <0.5 :
        return 0
    if score > 0.9 :
        return 1
    return 2
    

In [20]:
is_helpful(0.8)

2

In [40]:
is_helpful(0.2)

0

In [41]:
test_data['is_helpful'] = test_data['helpfulness_score'].apply(is_helpful)

In [42]:
test_data['is_helpful'].head()

0    1
1    1
2    1
3    1
4    1
Name: is_helpful, dtype: int64

In [43]:
test_data['review_length'] = test_data['reviewText'].apply(review_length)

In [44]:
test_data['review_length'].head()

0    458
1    346
2    210
3    134
4     95
Name: review_length, dtype: int64

In [45]:
preds_test = xgb_cl.predict(test_data[['overall','review_length']])

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [46]:
print(classification_report(test_data['is_helpful'], preds_test))

             precision    recall  f1-score   support

          0       0.29      0.81      0.42      3587
          1       0.67      0.86      0.75     17249
          2       0.00      0.00      0.00     11674

avg / total       0.38      0.55      0.44     32510



C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [49]:
print(confusion_matrix(test_data['is_helpful'], preds_test))

[[ 2919   668     0]
 [ 2414 14835     0]
 [ 4879  6795     0]]


In [ ]:
 EXPERIMENT 2

In [51]:
xgb_cl2 = xgb.XGBClassifier()

In [52]:
xgb_cl2.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
preds_test2=xgb_cl2.predict(test_data[['overall','review_length']])

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [54]:
print(confusion_matrix(test_data['is_helpful'],preds_test2))

[[ 2905   682     0]
 [ 2427 14822     0]
 [ 4862  6812     0]]


In [55]:
print(classification_report(test_data['is_helpful'], preds_test2))

             precision    recall  f1-score   support

          0       0.28      0.81      0.42      3587
          1       0.66      0.86      0.75     17249
          2       0.00      0.00      0.00     11674

avg / total       0.38      0.55      0.44     32510



C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Predicting for Review Data

In [21]:
review_df = pd.read_csv('Amazon_Latest_Data.csv')

In [23]:
len(review_df)

81275

In [25]:
review_df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime',
       'helpfulness_score', 'user_deviation', 'review_delay', 'stem_words',
       'lem_words', 'stem_sim_words', 'lem_sim_words'],
      dtype='object')

In [27]:
review_df['review_length'] = review_df['reviewText'].apply(review_length)

In [28]:
review_df['review_length'].head()

0    427
1    846
2    449
3     64
4    138
Name: review_length, dtype: int64

In [30]:
review_df['is_helpful'] = review_df['helpfulness_score'].apply(is_helpful)

In [31]:
test_data =review_df[['overall','review_length']]

In [32]:
preds = xgb_cl.predict(test_data)

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [37]:
print(classification_report(review_df['is_helpful'], preds))

             precision    recall  f1-score   support

          0       0.29      0.81      0.42      8962
          1       0.66      0.86      0.75     42988
          2       0.00      0.00      0.00     29325

avg / total       0.38      0.55      0.44     81275



C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
print(confusion_matrix(review_df['is_helpful'], preds))

[[ 7300  1662     0]
 [ 5969 37019     0]
 [12226 17099     0]]


In [39]:
helper_df= review_df[['reviewerID','asin','reviewText','overall','review_length']]

In [40]:
helper_df['help_pred_1'] = preds

C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
helper_df.to_csv('helper_df.csv',index=False)

### With Added Features 

In [ ]:
1. Difficult Words
2. Positive Words
3. Negative Words
4. Words Per Sentence 
5. smog index
6. coleman liau index
7. linsear write formula
8. sentence count


In [43]:
### get the text df
text_df = pd.read_csv('Text_Parameters.csv')

In [44]:
text_df.head()

,reviewText,flesch_reading_ease,helpfulness_score,smog_index,flesch_kincaid_grade,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,sentence_count,wps,review_length,pos_words,neg_words,pos_no,neg_no
0,"I'm a professional OTR truck driver, and I bou...",-336.56,0.800000,0.0,166.3,7.69,211.1,26.44,44,14.000000,176.92,-1th and 0th grade,1,427.000000,427,"['available', 'preferred']","['problem', 'refused', 'cheap', 'lost']",2,4
1,"Well, what can I say. I've had this unit in m...",-340.96,0.955556,0.0,165.9,9.72,210.8,26.37,94,8.666667,175.64,165th and 166th grade,2,423.000000,846,"['favorite', 'like', 'best', 'best', 'importan...","['strictly', 'slow', 'overwhelming', 'omission...",18,8
2,"Not going to write a long review, even thought...",85.52,0.900000,7.3,6.2,5.92,7.2,5.87,36,7.666667,13.01,5th and 6th grade,23,19.521739,449,"['well', 'pretty', 'fastest', 'excited', 'grea...","['trouble', 'lacking', 'mistakes', 'problem', ...",7,6
3,Quality was excellent. Instructions were clear...,66.94,0.789474,10.8,7.1,8.05,6.3,6.99,11,7.200000,14.00,6th and 7th grade,5,12.800000,64,"['clear', 'enough']",[],2,0
4,I checked around Amazon as well as some other ...,-43.22,0.444444,0.0,53.6,8.09,67.2,12.20,15,4.400000,61.55,53th and 54th grade,1,138.000000,138,['well'],"['damage', 'issue', 'hard']",1,3


In [46]:
df2['smog_index']= df2['reviewText'].swifter.apply(textstat.smog_index)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [03:39<00:00, 273.00it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
df2['flesch_kincaid_grade']= df2['reviewText'].swifter.apply(textstat.flesch_kincaid_grade)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [04:23<00:00, 227.50it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
df2['coleman_liau_index']= df2['reviewText'].swifter.apply(textstat.coleman_liau_index)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [02:04<00:00, 480.49it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df2['automated_readability_index']= df2['reviewText'].swifter.apply(textstat.automated_readability_index)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [02:08<00:00, 191.51it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
df2['dale_chall_readability_score']= df2['reviewText'].swifter.apply(textstat.dale_chall_readability_score)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [03:28<00:00, 138.77it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
df2['difficult_words']= df2['reviewText'].swifter.apply(textstat.difficult_words)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [01:13<00:00, 814.94it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
df2['linsear_write_formula']= df2['reviewText'].swifter.apply(textstat.linsear_write_formula)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [01:19<00:00, 756.40it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
df2['gunning_fog']= df2['reviewText'].swifter.apply(textstat.gunning_fog)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [03:17<00:00, 303.18it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
df2['text_standard']= df2['reviewText'].swifter.apply(textstat.text_standard)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [08:37<00:00, 115.87it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
df2['sentence_count']= df2['reviewText'].swifter.apply(textstat.sentence_count)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [01:18<00:00, 766.39it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
def get_words_per_sentence(text):
   return len(text.split())/textstat.sentence_count(text)
    

In [57]:
df2['wps']= df2['reviewText'].swifter.apply(get_words_per_sentence)

Pandas Apply: 100%|█████████████████████████████████████████████████████████████| 59989/59989 [01:22<00:00, 724.08it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
pos_file = open('opinion-lexicon-English/positive-words.txt','r')
neg_file = open('opinion-lexicon-English/negative-words.txt','r')

In [59]:
if pos_file.mode == 'r' :
    pos_content = pos_file.read()
    pos_list = pos_content.split('\n')
else :
    print('ERROR: could not read positive words!')

In [60]:
if neg_file.mode == 'r' :
    neg_content = neg_file.read()
    neg_list = neg_content.split('\n')
else :
    print('ERROR: could not read negative words!')

In [61]:
pos_list[-5:]

['youthful', 'zeal', 'zenith', 'zest', 'zippy']

In [62]:
neg_list[-5:]

['zaps', 'zealot', 'zealous', 'zealously', 'zombie']

In [63]:
pos_file.close()
neg_file.close()

In [64]:
def get_positive_words(text) :
    words = text.split()
    
    review_pos = []
    
    for word in words :
        if word in pos_list:
            review_pos.append(word)
            
    return review_pos
    

In [65]:
def get_negative_words(text) :
    words = text.split()
    
    review_neg = []
    
    for word in words :
        if word in neg_list:
            review_neg.append(word)
            
    return review_neg
    

In [66]:
df2['pos_words'] = df2['reviewText'].swifter.apply(get_positive_words)

Pandas Apply: 100%|██████████████████████████████████████████████████████████████| 59989/59989 [56:33<00:00, 35.38s/it]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [67]:
df2['neg_words'] = df2['reviewText'].swifter.apply(get_negative_words)

Pandas Apply: 100%|██████████████████████████████████████████████████████████████| 59989/59989 [45:15<00:00,  5.33it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
df2['pos_no'] = df2['pos_words'].swifter.apply(len)

Pandas Apply: 100%|██████████████████████████████████████████████████████████| 59989/59989 [00:00<00:00, 217284.42it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [69]:
df2['neg_no'] = df2['neg_words'].swifter.apply(len)

Pandas Apply: 100%|██████████████████████████████████████████████████████████| 59989/59989 [00:00<00:00, 224574.46it/s]
C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [71]:
df2.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime',
       'helpfulness_score', 'is_helpful', 'review_length', 'smog_index',
       'flesch_kincaid_grade', 'coleman_liau_index',
       'automated_readability_index', 'dale_chall_readability_score',
       'difficult_words', 'linsear_write_formula', 'gunning_fog',
       'text_standard', 'sentence_count', 'wps', 'pos_words', 'neg_words',
       'pos_no', 'neg_no'],
      dtype='object')

In [239]:
X = df2[['review_length','overall', 'wps', 'pos_no' ,'neg_no', 'linsear_write_formula']]
len(X.columns)

6

In [240]:
y =df2['is_helpful']

In [241]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=101)

In [242]:
xgb_cl_3 = xgb.XGBClassifier()

In [243]:
xgb_cl_3.fit(X_train,y_train, verbose=True)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [244]:
predictions=xgb_cl_3.predict(X_test)

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [245]:
print(confusion_matrix(y_test,predictions))

[[ 7524  2311]
 [ 1911 12250]]


In [246]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.80      0.77      0.78      9835
          1       0.84      0.87      0.85     14161

avg / total       0.82      0.82      0.82     23996



In [247]:
test_data.head()

,review_length,overall,unixReviewTime,wps,pos_no,neg_no,linsear_write_formula
0,427,1.0,1290643200,427.000000,2,4,14.000000
1,846,3.0,1283990400,423.000000,18,8,8.666667
2,449,2.0,1290556800,19.521739,7,6,7.666667
3,64,5.0,1286236800,12.800000,2,0,7.200000
4,138,1.0,1291161600,138.000000,1,3,4.400000


In [253]:
test_data = review_df[['review_length', 'overall']]
test_data=test_data.join(text_df[['wps', 'pos_no', 'neg_no', 'linsear_write_formula']])

In [254]:
test_data.head()

,review_length,overall,wps,pos_no,neg_no,linsear_write_formula
0,427,1.0,427.000000,2,4,14.000000
1,846,3.0,423.000000,18,8,8.666667
2,449,2.0,19.521739,7,6,7.666667
3,64,5.0,12.800000,2,0,7.200000
4,138,1.0,138.000000,1,3,4.400000


In [255]:
preds_test = xgb_cl_3.predict(test_data)

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [263]:
print(classification_report(review_df['is_helpful'], preds_test))

C:\Users\Trishul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.29      0.82      0.43      8962
          1       0.66      0.86      0.75     42988
          2       0.00      0.00      0.00     29325

avg / total       0.38      0.55      0.44     81275



In [261]:
print(confusion_matrix(review_df['is_helpful'], preds_test))

[[ 7332  1630     0]
 [ 5841 37147     0]
 [12062 17263     0]]


In [264]:
helper_df['help_pred_2'] = preds_test

C:\Users\Trishul\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [265]:
helper_df.to_csv('helper_df.csv',index=False)

In [266]:
helper_df.head()

,reviewerID,asin,reviewText,overall,review_length,help_pred_1,help_pred_2
0,AMO214LNFCEI4,0528881469,"I'm a professional OTR truck driver, and I bou...",1.0,427,0,0
1,A3N7T0DY83Y4IG,0528881469,"Well, what can I say. I've had this unit in m...",3.0,846,1,1
2,A1H8PY3QHMQQA0,0528881469,"Not going to write a long review, even thought...",2.0,449,0,0
3,A3QH8VQDE7HZCR,0972683275,Quality was excellent. Instructions were clear...,5.0,64,1,1
4,A38FGQVJM18OWV,0972683275,I checked around Amazon as well as some other ...,1.0,138,0,0
